<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project Check-in</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project Check-in
------

Group Name: SAL
-----

Student Names
----

1. Lisa Chua
2. Alaa A Latif
3. Shane Buchanan 

Load Data
-----

In [44]:
reset -fs

In [133]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SMOTENC

In [108]:
t = (df.dtypes == 'object')
trues = [i for i, x in enumerate(t) if x][:-1]
trues

[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [82]:
def make_pipeline(classifier=None):
    "Create a single pipeline that processes the data and then fits the regressor." 
    
    # YOUR CODE HERE
    numeric_features     = ['age', 'campaign', 'cons.price.idx', 'pdays', 'previous', 'emp.var.rate', 'cons.conf.idx',
                            'euribor3m', 'nr.employed']
    categorical_features = ['job', 'education', 'default', 'housing',
                            'loan', 'campaign', 'poutcome', 'marital', 'contact', 'month', 'day_of_week']
    
    categorical_transformer = Pipeline(steps=[
    ('impute', impute.SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])
    
    numeric_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='median')),
    ('scaler', preprocessing.StandardScaler())])
    
    
    
    preprocessor = compose.ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', classifier)])
    return pipeline

In [83]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('bank-additional-full.csv', delimiter=';')

In [84]:
# Features data.
X = df.drop('y', axis=1)

In [85]:
y = df['y']

In [86]:
# Train-test-split on data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [127]:
(y_train == 'yes').sum()

3697

### Rebalance the data set

In [87]:
def up_sample(X, y):
    rows_idxs_no = y == 'no'
    rows_idxs_yes = y == 'yes'
    df_yes = pd.concat((X.loc[rows_idxs_yes], y.loc[rows_idxs_yes]), axis = 1)
    df_no = pd.concat((X.loc[rows_idxs_no], y.loc[rows_idxs_no]), axis = 1)
    df_yes_upsampled = df_yes.sample(int(len(df_yes)*5), replace=True)
    df_upsampled = pd.concat([df_no, df_yes_upsampled], axis=0)
    return df_upsampled.drop('y', axis=1), df_upsampled['y']

# K Fold Cross Validation

In [88]:
kf = KFold(n_splits=5, shuffle=True)

In [152]:
smt = SMOTENC(trues)
X_train_up, y_train_up = smt.fit_sample(X_train, y_train)
X_train_up = pd.DataFrame(X_train_up, columns=X.columns)
y_train_up = pd.Series(y_train_up)
for train_index, test_index in kf.split(X_train_up):
    X_train_up1, X_val_up = X_train_up.iloc[train_index], X_train_up.iloc[test_index]
    y_train_up1, y_val_up = y_train_up.iloc[train_index], y_train_up.iloc[test_index]
    #clf = DecisionTreeClassifier()
    #clf = LogisticRegression()
    clf = RandomForestClassifier()
    pipeline = make_pipeline(clf)
    pipeline.fit(X_train_up1, y_train_up1)
    y_pred = pipeline.predict(X_val_up)
    print(f1_score(y_val_up, y_pred, average='weighted'))

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9016033447205567


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8975154243767558


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9016436477150052


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8974002683504024


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9003777888435691


Fit scikit-learn model
----

In [139]:
# # Initialize dtree and fit.
# clf = DecisionTreeClassifier()
# pipeline = make_pipeline(clf)

## Predict

In [140]:
# # Train Estimator
# pipeline.fit(X_train, y_train);

In [153]:
y_pred = pipeline.predict(X_test)

In [154]:
confusion_matrix(y_test, y_pred)

array([[6856,  439],
       [ 562,  381]])

Evaluation Metric
----

In [155]:
# Use F1 score as metric since we have an unbalanced data set.
f1_score(y_test, y_pred, average='weighted')

0.8747591104900356

In [156]:
accuracy_score(y_test, y_pred)

0.8784899247390143